In [1]:
#!pip3 install tqdm
from post_processing import *
from const import ROOT
from const import *
from PIL import ImageOps
def tint_image(src, color="#FFFFFF"):
    src.load()
    r, g, b, alpha = src.split()
    gray = ImageOps.grayscale(src)
    result = ImageOps.colorize(gray, (0, 0, 0, 0), color) 
    result.putalpha(alpha)
    return result
import pandas as pd
#ROOT='/Users/alexanderliao/data/'
ROOT='/home/alexanderliao/data/Kaggle/competitions/tgs-salt-identification-challenge/'
#ROOT='/data/kaggle/salt/'
train_path = os.path.join(ROOT, 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]
test_path = os.path.join(ROOT, 'test')
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]


def is_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def down_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(50,101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[50,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def find_cols(img):
    row = img[100,:]
    #print(row)
    indx1 = -1
    indx2 = -1
    indx3 = -1
    indx4 = -1
    logic = True
    logic2 = True
    for i in range(len(row)):
        #print(row[i])
        if(row[i]==1):
            if indx1==-1 and indx2 == -1:
                indx1=i
            #print(indx1,indx2)
            if logic2 and indx1!=-1 and indx2 !=-1:
                indx3=i
                logic2=False
        else:
            if logic and indx1!=-1 and indx3 ==-1:
                indx2=i
                logic=False
            if indx3!=-1:
                indx4=i
    indx2=indx2-1
    indx4=indx4-1
    if indx4==-2: indx4=100
    return indx1, indx2, indx3, indx4

def find_rows(img):
    row = img[:,0]
    #print(row)
    indx1 = -1
    indx2 = -1
    indx3 = -1
    indx4 = -1
    logic = True
    logic2 = True
    for i in range(len(row)):
        #print(row[i])
        if(row[i]==1):
            if indx1==-1 and indx2 == -1:
                indx1=i
            #print(indx1,indx2)
            if logic2 and indx1!=-1 and indx2 !=-1:
                indx3=i
                logic2=False
        else:
            if logic and indx1!=-1 and indx3 ==-1:
                indx2=i
                logic=False
            if indx3!=-1:
                indx4=i
    indx2=indx2-1
    indx4=indx4-1
    if indx4==-2: indx4=100
    return indx1, indx2, indx3, indx4

def vertical_sweep(img,col1,col2):
    img2=img.copy()
    img2[:,col1:col2+1]=1
    return img2

def vertical_clear(img,col1,col2):
    img2=img.copy()
    img2[:,0:col1]=0
    img2[:,col2:]=0
    return img2

def horizontal_clear(img,col1,col2):
    img2=img.copy()
    img2[0:col1,:]=0
    img2[col2:,:]=0
    return img2

def ul_empty(img1,img2):
    if img1[:,0:10].sum()==0 and img2[0:10,:].sum()==0: return True
    return False
    
def end_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(90,101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[90,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def up_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(25):
        if img[i,:].sum()==0: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0
dicts = pickle.load( open( "linked_list.p", "rb" ) )
def id_to_img(id):
    if id in train_file_list:
        str="train"
    else:
        str="test"
    img = Image.open(ROOT+str+"/images/"+id+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    return arr

def id_to_mask(id):
    img = Image.open(ROOT+"train/masks/"+id+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    return arr
import tensorflow as tf
df = pd.read_csv("0871.csv")
def column(matrix, i):
    return [row[i] for row in matrix]
def rule_empty1(col_curr,j):
    return id_to_img(col_curr[j-1]).sum()!=0
def isnt_empty(img):
    return img.sum()>0 and img.sum()<2600000
def rule_ne(col_curr,j):
    if col_curr[j-1] in test_file_list: return True
    else:
        mask=id_to_mask(col_curr[j-1])
        if not(is_vertical(mask)):
            if down_vertical(mask): return True
            else: return False
    return True

def deep_rejection(col_curr,j,vert_mask):
    if col_curr[j-1] in test_file_list: return True
    else:
        mask=id_to_mask(col_curr[j-1])
        if not(is_vertical(mask)):
            if down_vertical(mask): return True
            else: return False
    return True

def rle_decode(rle_mask,size):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(size*size, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(size,size)

def get_iou_vector(A, B):
    # Numpy version
    
    batch_size = A.shape[0]
    metric = 0.0
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # deal with empty mask first
        if true == 0:
            metric += (pred == 0)
            continue
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        
        # iou metrric is a stepwise approximation of the real iou over 0.5
        iou = np.floor(max(0, (iou - 0.45)*20)) / 10
        
        metric += iou
        
    # teake the average over all images in batch
    metric /= batch_size
    return metric


def iou_metric(label, pred):
    # Tensorflow version
    return tf.py_func(get_iou_vector, [label, pred > 0.5], tf.float64)

def LB_improvement(new_code, old_code):
    strs=""
    for string in new_code:
       strs=strs+str(string)+" " 
    img_new = np.transpose(rle_decode(strs,101))
    if img_new.sum() == 0:
        return 1
    if len(old_code)==0: return 1
    else:
        old_code = old_code.values[0]
    
    #img_old = rle_decode(old_code[old_code.find(" "):],101)
    if type(old_code) is float:
        return 1
    else:
        img_old = np.transpose(rle_decode(old_code,101))
        #print(img_new.shape)
        #print(img_old.shape)
        #iou = iou_metric(img_new,img_old)
        #print(iou)
        difference = 1-iou_metric(img_new,img_old)
        return difference
    
def LB_improvement2(new_code, old_code):
    img_new = np.transpose(rle_decode(new_code,101))
    if img_new.sum() == 0:
        return 1
    if len(old_code)==0: return 1
    else:
        old_code = old_code.values[0]
    
    #img_old = rle_decode(old_code[old_code.find(" "):],101)
    if type(old_code) is float:
        return 1
    else:
        img_old = np.transpose(rle_decode(old_code,101))
        #print(img_new.shape)
        #print(img_old.shape)
        #iou = iou_metric(img_new,img_old)
        #print(iou)
        difference = 1-iou_metric(img_new,img_old)
        return difference
    
def discrepancy(img):
    diffs=[]
    #if img.sum()==0: return False
    for i in range(101):
        #if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    return discrepancy

/home/alexanderliao/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def create_polygon_dr(down,right):
    indx1 = np.where(down==1)[0][0]
    indx2 = np.where(right==1)[0][0]
    arr = np.zeros((202, 202))
    rr, cc = draw.circle_perimeter(101-indx1, 101-indx2, radius=80, shape=arr.shape)
    arr[rr, cc] = 1
    reversed_arr = arr[::-1]

In [3]:
a=np.array([0,0,1])
np.where(a==1)[0][0]

2

In [4]:
#df = pd.read_csv("final/ocnet256_resnet256_0450350101_0871empty_leak4++_stage2+++.csv")

In [5]:
dfne = pd.read_csv("final/ocnet256_resnet256_0450350101.csv")
#df.sort_values(by=['id'])
#remove= [1,5,9,13,15,18,21,25,29,33,43,45,57,62,65,74,107,110,136,150,7,42]
remove = range(1,151)

In [6]:
#remove= [1,5,9,13,15,18,21,25,29,33,43,45,57,62,65,74,107,110,136,150,7,42,4]
#        y y y  y  y  y  y  y y  y  y   y  y  y  y y   y   y   y   y  n  y
extra=[20,34,39,47]
other_half=[13,43,65,110,136,150,7,42,4]
add=[1,5,6,10,12,13,14,18,19,23,33,34,46,47,53,56,58,67,77,78,81,102,106]

In [7]:
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3+.csv")
#df = pd.read_csv("final/test.csv")
per_im=0
RULE1=True
RULE2=True
RULE3=True
RULE4=True
RULE5=True
rule1=0
rule2=0
rule3=0
rule4=0
ct=0
for i in add:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    arr = np.transpose(rle_decode(string,101))
#========================================================================================#

            if not(isnt_empty(arr)) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)

                if index1 < len(col_curr)-2 and index1 > 2 and index2 < len(row)-2 and len(col_curr)>4:
                    imgs=[]
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    row_above = list[index1-2]
                    imgs.append(row_above[index2-1])
                    imgs.append(row_above[index2-2])
                    imgs.append(row_above[index2-3])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()==0: logic1=False
                    
                    imgs=[]
                    imgs.append(row_above[index2+1])

                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic2=True
                    for mask in masks:
                        if mask.sum()!=0: logic2=False
                    
                    #if(i==23): 
                    #    print(index1,index2)
                    #    print(logic1, logic2)
                    
                    if logic1 and logic2 and col in test_file_list:
                        new_code = df.loc[df['id'] == col_curr[index1-1]]['rle_mask'].values[0]
                        #print(index2,len(col_curr))
                        for j in range(index1, len(col_curr)):
                            cand = col_curr[j]
                        
                            df.loc[df['id'] == cand,'rle_mask']=new_code
                            ct=ct+1
                            per_im=per_im+1
                            rule4=rule4+1
                            visited.append(col)

                            print("-----------")
                            print("right boundary")
                            print("-----------")
                            #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                            print("reference {}: row {}, col {}".format(cand,j,row.index(col)))


#========================================================================================#
#========================================================================================#

            if not(isnt_empty(arr)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
                #print(index2)
                if index1 <= len(col_curr)/2 and index1 > 2 and index2 < len(row)-2 and len(col_curr)>8:
                    imgs=[]
                    imgs.append(col_curr[index1-2])
                    imgs.append(col_curr[index1-3])
                    imgs.append(col_curr[index1-4])
                    imgs.append(col_curr[index1+2])
                    imgs.append(col_curr[index1+3])
                    imgs.append(col_curr[index1+4])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()==0: logic1=False
                    
                    
                    imgs=[]
                    col_right = column(list,row.index(col)+1)
                    imgs.append(col_right[index1])
                    imgs.append(col_right[index1+1])
                    imgs.append(col_right[index1-1])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic2=True
                    for mask in masks:
                        if mask.sum()!=0: logic2=False
                    

                    #if i==33:
                    #    print(index1,index2)
                    #    print(logic1, logic2)
                    if logic1 and logic2:
                        new_code = df.loc[df['id'] == col_curr[index1-2]]['rle_mask'].values[0]
                        if i==33:
                            print(index1)
                        for j in range(index1-2, len(col_curr)):
                            cand = col_curr[j]
                            test=df.loc[df['id'] == cand]['rle_mask'].values
                            if j ==3 or j==5: print(test)
                            if (len(test)==0 or type(test[0]) is float) and cand in test_file_list:
                                df.loc[df['id'] == cand,'rle_mask']=new_code
                                ct=ct+1
                                per_im=per_im+1
                                rule4=rule4+1
                                visited.append(col)

                                print("-----------")
                                print("connect up and down: gaps 3")
                                print("-----------")
                                #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                                print("reference {}: row {}, col {}".format(cand,j,row.index(col)))


#========================================================================================#
#========================================================================================#

            if not(isnt_empty(arr)) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)

                if index1 < 2 and index2 < len(row)-2 and len(col_curr)>6:
                    imgs=[]
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1+2])
                    imgs.append(col_curr[index1+3])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()!=0: logic1=False
                    
                    imgs=[]
                    col_right = column(list,row.index(col)+1)
                    row_above = list[index1-1]
                    imgs.append(col_curr[index1+5])
                    imgs.append(row_above[index2-1])
                    imgs.append(row_above[index2-2])
                    imgs.append(col_curr[index1+4])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic2=True
                    for mask in masks:
                        if mask.sum()==0: logic2=False
                    
                    #if i==12: 
                    #    print(index1,index2)
                    #    print(logic1, logic2)
                    
                    if logic1 and logic2:
                        new_code = df.loc[df['id'] == col_curr[index1+5]]['rle_mask'].values[0]
                        for j in range(index1, len(col_curr)):
                            
                            cand = col_curr[j]
                            test=df.loc[df['id'] == cand]['rle_mask'].values
                            
                            if len(test)==0 or type(test[0]) is float:
                                df.loc[df['id'] == cand,'rle_mask']=new_code
                                ct=ct+1
                                per_im=per_im+1
                                rule4=rule4+1
                                visited.append(col)

                                print("-----------")
                                print("connect up and down: gaps 3 #2")
                                print("-----------")
                                #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                                print("reference {}: row {}, col {}".format(cand,j,row.index(col)))


#========================================================================================#
#========================================================================================#

            if not(isnt_empty(arr))  and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)

                if index1>2 and index1 < len(col_curr)-2 and index2 < len(row)-4 and len(col_curr)>6 and len(row)>4:
                    imgs=[]
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    row_above=list[index1-3]
                    imgs.append(row_above[index2+3])
                    imgs.append(row_above[index2+2])
                    imgs.append(row_above[index2+1])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()==0: logic1=False
                  
                    imgs=[]
                    #col_right = column(list,row.index(col)+1)
                    #row_above = list[index1-1]
                    imgs.append(col_curr[index1+1])
                    imgs.append(col_curr[index1+2])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic2=True
                    for mask in masks:
                        if mask.sum()!=0: logic2=False
                    
                    #if i==19: 
                    #    print(index1,index2)
                    #    print(logic1, logic2)
                    
                    if logic1 and logic2:
                        new_code = df.loc[df['id'] == col_curr[index1-1]]['rle_mask'].values[0]
                        for j in range(index1, len(col_curr)):
                            
                            cand = col_curr[j]
                            test=df.loc[df['id'] == cand]['rle_mask'].values
                            #print(test)
                            if len(test)==0 or type(test[0]) is float and cand in test_file_list:
                                df.loc[df['id'] == cand,'rle_mask']=new_code
                                ct=ct+1
                                per_im=per_im+1
                                rule4=rule4+1
                                visited.append(col)

                                print("-----------")
                                print("left boundary")
                                print("-----------")
                                #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                                print("reference {}: row {}, col {}".format(cand,j,row.index(col)))


#========================================================================================#
#========================================================================================#

            if not(isnt_empty(arr))  and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)

                if index1>2 and index1 < len(col_curr)-3 and index2>1 and index2 < len(row)-2 and len(col_curr)>6 and len(row)>4:
                    imgs=[]
                    imgs.append(col_curr[index1-1])
                    imgs.append(col_curr[index1-2])
                    imgs.append(col_curr[index1-3])
                    row_above=list[index1-3]
                    imgs.append(row_above[index2-1])
                    imgs.append(row_above[index2+2])
                    imgs.append(row_above[index2+1])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic1=True
                    for mask in masks:
                        if mask.sum()==0: logic1=False
                  
                    imgs=[]
                    #col_right = column(list,row.index(col)+1)
                    #row_above = list[index1-1]
                    imgs.append(col_curr[index1+1])
                    imgs.append(col_curr[index1+2])
                    imgs.append(col_curr[index1+3])
                    masks=[]
                    
                    for img in imgs:
                        if img in train_file_list:
                            masks.append(id_to_mask(img)/255)
                        else:
                            string=df[df['id']==img]['rle_mask'].values[0]
                            if type(string) is float:
                                masks.append(np.zeros((101,101)))
                            else:
                                masks.append(np.transpose(rle_decode(string,101)))
                    logic2=True
                    for mask in masks:
                        if mask.sum()!=0: logic2=False
                    
                    #if i==47: 
                    #    print(index1,index2)
                    #    print(logic1, logic2)
                    
                    if logic1 and logic2:
                        new_code = df.loc[df['id'] == col_curr[index1-1]]['rle_mask'].values[0]
                        for j in range(index1, len(col_curr)):
                            
                            cand = col_curr[j]
                            test=df.loc[df['id'] == cand]['rle_mask'].values
                            #print(test)
                            if len(test)==0 or type(test[0]) is float and cand in test_file_list:
                                df.loc[df['id'] == cand,'rle_mask']=new_code
                                ct=ct+1
                                per_im=per_im+1
                                rule4=rule4+1
                                visited.append(col)

                                print("-----------")
                                print("final rule")
                                print("-----------")
                                #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                                print("reference {}: row {}, col {}".format(cand,j,row.index(col)))


#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#

#sess = tf.Session()
print("Stage 3")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))

-----------
connect up and down: gaps 3 #2
-----------
reference 37e8e0a012: row 1, col 4
-----------
connect up and down: gaps 3 #2
-----------
reference 2cb5045cba: row 2, col 4
-----------
connect up and down: gaps 3 #2
-----------
reference 5aa1bc199b: row 3, col 4
-----------
connect up and down: gaps 3 #2
-----------
reference 4aa6feb688: row 4, col 4
image 12: 4
-----------
right boundary
-----------
reference 380e59ba25: row 3, col 3
-----------
right boundary
-----------
reference f869349ffe: row 4, col 3
-----------
right boundary
-----------
reference 56877ea14c: row 5, col 3
-----------
right boundary
-----------
reference 80a60e50be: row 6, col 3
image 23: 4
4
[nan]
-----------
connect up and down: gaps 3
-----------
reference ef04597f79: row 3, col 0
[nan]
-----------
connect up and down: gaps 3
-----------
reference 58a4b8ec4a: row 5, col 0
image 33: 2
-----------
final rule
-----------
reference 2b66c196d7: row 4, col 2
-----------
final rule
-----------
reference 930e7

In [8]:
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3++.csv",index=False)
#df.to_csv("final/test.csv",index=False)
other_half=[13,43,65,110,136,150,7,42,4]

In [ ]:
add=[1,5,6,10,12,13,14,18,19,23,33,34,46,53,56,58,67,77,78,81,102,106]
#   n  n n  n  n n  n   n  n  y  y  n  n  n  n  n  n  n  n  n  n   n

In [2]:
import pickle
preds=pickle.load(open("final/stable_all_sigmoids.p","rb"))

threshold = 0.45
binary_prediction = (preds > threshold).astype(int)


def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

all_masks = []
for p_mask in list(binary_prediction):
    p_mask = rle_encoding(p_mask)
    all_masks.append(' '.join(map(str, p_mask)))



submit = pd.DataFrame([test_file_list, all_masks]).T
submit.columns = ['id', 'rle_mask']
submit.to_csv('final/100_stable_models_weighted.csv', index = False)


In [3]:
del preds